<a href="https://colab.research.google.com/github/gitconnoisseur/toxicity-model/blob/master/ecfp_xgb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyTDC
!pip install deepchem
!pip install autokeras
!pip install --upgrade numpy

In [1]:
#import autogluon as ag
from tdc.single_pred import Tox
import deepchem
import xgboost

import numpy as np
import pandas as pd
import tensorflow as tf
import autokeras as ak

from tdc.benchmark_group import admet_group
group = admet_group(path = 'data/')

Found local copy...


In [3]:
predictions_list = []
metric = deepchem.deepchem.metrics.Metric(deepchem.deepchem.metrics.mean_absolute_error)

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('LD50_Zhu') 
    # all benchmark names in a benchmark group are stored in group.dataset_names
    predictions = {}
    name = benchmark['name']
    train_val, test = benchmark['train_val'], benchmark['test']
    train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
    
        # --------------------------------------------- # 
        #  Train your model using train, valid, test    #
        #  Save test prediction in y_pred_test variable #
        # --------------------------------------------- #
    

    featurizer = deepchem.deepchem.feat.WeaveFeaturizer()
    f_train = featurizer.featurize(train.iloc[:,1].to_list())
    f_valid = featurizer.featurize(valid.iloc[:,1].to_list())

    train_data = deepchem.deepchem.data.NumpyDataset(X=f_train, y=np.array(train.iloc[:,2]), ids=np.array(train.iloc[:,1].to_list()))
    valid_data = deepchem.deepchem.data.NumpyDataset(X=f_valid, y=np.array(valid.iloc[:,2]), ids=np.array(valid.iloc[:,1].to_list()))

    reg = deepchem.deepchem.models.WeaveModel(n_tasks=1, mode="regression", batch_normalize_kwargs={"trainable": False})
    callback = deepchem.deepchem.models.ValidationCallback(valid_data, 1000, metric)
    reg.fit(train_data, nb_epoch=100, callbacks=callback)

    testFeatures = np.array(featurizer.featurize(test.iloc[:,1].to_list()))

    y_pred_test = reg.predict_on_batch(X=testFeatures)

    print("predictions for run #")
    print(seed)
    print(y_pred_test[0:5])
        
    predictions[name] = y_pred_test
    predictions_list.append(predictions)

print("Prediction List:")
print(predictions_list)

generating training, validation splits...
100%|██████████| 5907/5907 [00:01<00:00, 3894.79it/s]


KeyboardInterrupt: 

In [5]:
for i in predictions_list:
    print(group.evaluate(i))

group.evaluate_many(predictions_list)

{'ld50_zhu': {'mae': 0.68}}


ValueError('Must have predictions from at least 5 runs for leaderboard submission')

In [6]:
# multiply mae by negative one so it can be maximized
hyperOptMetric = deepchem.deepchem.metrics.Metric(lambda *args: -1.0 * deepchem.deepchem.metrics.mean_absolute_error(*args), mode="regression")

def model_builder(**model_params):
  return deepchem.deepchem.models.WeaveModel(n_tasks=1, mode="regression", batch_normalize_kwargs={"trainable": False})

#optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

optimizer = deepchem.deepchem.hyper.GaussianProcessHyperparamOpt(model_builder)

params_dict = {"n_pair_feat": 14, "n_hidden": 50}

best_model, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict, train_data, valid_data, hyperOptMetric, search_range=5)

Evaluation 	 Proposed point 	  Current eval. 	 Best eval.


init   	 [130. 329.  17.]. 	  -0.7555424053672201 	 -0.6375722658437386
init   	 [  4. 285.  17.]. 	  -0.6375722658437386 	 -0.6375722658437386
init   	 [ 48. 364.   7.]. 	  -0.7049368234474378 	 -0.6375722658437386


1      	 [137.  40.   4.]. 	  -0.6186904076187795 	 -0.6186904076187795


2      	 [ 38. 448.   3.]. 	  -0.5716399556767795 	 -0.5716399556767795


3      	 [ 73. 401.  19.]. 	  -0.5750580500485288 	 -0.5716399556767795


4      	 [ 67. 306.  14.]. 	  -0.5769406830009493 	 -0.5716399556767795


5      	 [ 4. 21.  0.]. 	  -0.5816318836108919 	 -0.5716399556767795


6      	 [ 25. 277.   1.]. 	  -0.5791916413429787 	 -0.5716399556767795


KeyboardInterrupt: 